In [226]:
import numpy as np
import pandas as pd

In [227]:
df=pd.read_csv('train_titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [228]:
#from sklearn.preprocessing import LabelEncoder
#encoder=LabelEncoder()
#df['PassengerId']=encoder.fit_transform(df['PassengerId'])
#df.head()

In [229]:
# droping some columns directly beacuse the dont contribute much

df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [230]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [214]:
# filling missing values in age and embarked columns
# onehotencoding on sex and embarked

In [231]:
X=df.iloc[:,1:]
y=df.iloc[:,0]

In [232]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [233]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [234]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)

In [235]:
print(X_train.head())
print(y_train.head())

     Pclass     Sex    Age  SibSp  Parch      Fare Embarked
298       1    male    NaN      0      0   30.5000        S
884       3    male  25.00      0      0    7.0500        S
247       2  female  24.00      0      2   14.5000        S
478       3    male  22.00      0      0    7.5208        S
305       1    male   0.92      1      2  151.5500        S
298    1
884    0
247    1
478    0
305    1
Name: Survived, dtype: int64


In [236]:
print(X_test.head())
print(y_test.head())

     Pclass     Sex   Age  SibSp  Parch     Fare Embarked
709       3    male   NaN      1      1  15.2458        C
439       2    male  31.0      0      0  10.5000        S
840       3    male  20.0      0      0   7.9250        S
720       2  female   6.0      0      1  33.0000        S
39        3  female  14.0      1      0  11.2417        C
709    1
439    0
840    0
720    1
39     1
Name: Survived, dtype: int64


In [237]:
# for missing values use simple imputer from sklearn because we can pickel that

# we use column transformer because it is easy to do and instaed of manually doing we can do by this


from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer


In [238]:
trf1= ColumnTransformer([
    ('impute1',SimpleImputer(),[2]),
    ('impute2',SimpleImputer(strategy='most_frequent'),[6])

], remainder='passthrough')

                                             #beacuse when we apply column transformer dataframe convert to numpy array so 2 is index of age
# impute is name of transformation then object that should be used then index of column and all this is in tuple
#default of simpleimputer() is mean so missing values replaced by mean

# remainder is for what to do whith other columns so two values drop or passthrough

In [239]:
#trf1.fit_transform(X_train)

In [240]:
#trf1.named_transformers_

In [241]:
#trf1.named_transformers_['impute1'].statistics_

In [242]:
#trf1.named_transformers_['impute2'].statistics_

In [243]:
from sklearn.preprocessing import OneHotEncoder
trf2= ColumnTransformer([
    ('one',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])

])

#onehotencoder returns sparse array so to convert to numpy array do sparse=False
#and handle_unkown='ignore' means like there is one S W category in training example but
#got another category in test example like Q so it wil handle that making Q value 0 computing such a way

In [244]:
from xgboost import XGBClassifier

clf=XGBClassifier()

In [245]:
# now we create pipleline doing above 3 steps we define

from sklearn.pipeline import Pipeline


In [246]:
pipe=Pipeline([
    ('step1',trf1),
    ('step2',trf2),
    ('step3',clf)
])

In [247]:
pipe.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute1', SimpleImputer(),
                                                  [2]),
                                                 ('impute2',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('step2',
                 ColumnTransformer(transformers=[('one',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('step3',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               c...
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=None, ...))])

In [250]:
y_pred=pipe.predict(X_test)

In [251]:
from sklearn.metrics import accuracy_score
print('accuracy= ',accuracy_score(y_test,y_pred) )

accuracy=  0.6367713004484304


In [252]:
# we can instead of importing diff diff things can directly pickle transformer, this is adv.

In [253]:
import pickle

In [254]:
pickle.dump(pipe,open('pipe.pkl','wb'))

In [256]:
pipe=pickle.load(open('pipe.pkl','rb'))

In [259]:
pipe.predict(np.array([[1   , 'male'  ,  31   ,   0   ,   0  , 30.5000  ,'S']],dtype=object))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([1])